In [ ]:
from vpython import *

Just run the notebook and the demonstration will appear above in the output of the first cell.

In [ ]:
# ---------------------------------------------------------
# PARAMETER DECLARATIONS - and boy there are a lot of them!
# ---------------------------------------------------------
#
# Timestep information
t = 0.0
dt = 4.0e-3
ticks_per_s = int(2/dt)
#
# These are used to skip visualization frames
fps = 24 
ticks_per_frame = int(ticks_per_s/fps)

# Physical constants
#
chi = 0.8 # static susceptibility
h_static_0 = 5.0 # static magnetic field
h_RF_0 = 0.1*h_static_0 # RF amplitude 
gamma = 1.2 # gyromagnetic ratio
omega0 = gamma*h_static_0 # nominal resonant rotation frequency
omega1 = gamma*h_RF_0 # nominal RF rotation frequency
T0 = 2.0 # relaxation time, in units of 1/(f_1)


# 
# Magnetization vector initialization and creation
h0 = vector(0.,0.,h_static_0)
m0 = chi*h0
# Image objects of magnetization and field
#
M = arrow(pos=vector(0,0,0), axis=m0, shaftwidth=0.3, color=color.blue)
B = arrow(pos=vector(0,0,0), axis=h0, shaftwidth=0.3, color=color.green)

# -----------------------------------------------
# FUNCTION DEFINITIONS
# -----------------------------------------------
#

# Set up scene to give up = z-axis, looking down at x-y plane
#
def set_scene(myscene=scene):
    myscene.height = 400
    myscene.width = 400
    myscene.range = 12 # meters
    myscene.autoscale = False
    myscene.background = color.white
    myscene.forward = vector(-1,-0.5,-0.3)
    myscene.up = vector(0,0,1)
    myscene.align = "left"
    return myscene

#
# Pause/run button
def Run(b):
    global running
    running = not running
    if running: b.text = "Pause"
    else: b.text = "Run"
    
#
# Reset M to start button
def Reset_M():
    global m
    m = m0
    
#
# Clear the plot
def Clear_plot():
    global f1, t
    t = 0.0
    f1.data = [[t,0.0]]

# Stop/Start plotting    
def Stop_plot(b):
    global plotting
    plotting = not plotting
    if plotting: b.text = "Stop plot"
    else: b.text = "Start plot"

#
# Switch to rotating frame button
def Rotate_frame(b):
    global rotating
    rotating = not rotating
    if rotating: b.text = "Rotating frame"
    else: b.text = "Lab frame"
#
# Turn on/off RF
def Pulse_start(b):
    global pulsing
    pulsing = not pulsing
    if pulsing: b.text = "<b>RF ON</b>"
    else: b.text = "  RF  "

#
# Switch between linear and rotating RF field
def RF_select():
    global RFtype
    RFtype = RF_menu.selected
    
def RF_function(phi, RF_type='rotating'):
    if (RF_type == 'rotating'):
        return sl_h1.value*h_static_0*cos(phi), sl_h1.value*h_static_0*sin(phi)
    elif (RF_type == 'linear'):
        return 2.0*sl_h1.value*h_static_0*cos(phi), 0.0
    else:
        return 0.0, 0.0
#
# Set h0 and h1 entry boxes.
def Set_h1(h1_mag):
    wt_h1.text = '{:1.3f}'.format(h1_mag.value)

def Set_h0(h0_mag):
    wt_h0.text = '{:1.3f}'.format(h0_mag.value)

def Set_Delta_omega(domega):
    wt_dW.text = '{:1.3f}'.format(domega.value)
    
#
# Set T (T1 and T2 are the same) in units of 1/f1    
# T_menu = menu( choices=['1.0','4.0','10.0','100.0','1000.0'], bind=T_select )
def T_select():
    global T
    Tt = float(T_menu.selected)
    T = Tt/log(2)

#
# RF angular frequency
def omega():
    return omega0*(1+sl_dW.value)
    
#
# Draw coordinate axes
def draw_axes():
    xaxis = cylinder(pos=vector(-10,0,0), axis=vector(20,0,0), radius=0.05, color=color.gray(0.7) )
    xlabel = label(pos=vector(10.3,0,0), text='x', box=False )
    yaxis = cylinder(pos=vector(0,-10,0), axis=vector(0,20,0), radius=0.05, color=color.gray(0.7) )
    ylabel = label(pos=vector(0,10.3,0), text='y', box=False )
    zaxis = cylinder(pos=vector(0,0,-10), axis=vector(0,0,20), radius=0.05, color=color.gray(0.7) )
    zlabel = label(pos=vector(0,0,10.3), text='z', box=False )

In [ ]:
# -----------------------------------------------
# MAIN PROGRAM EXECUTION
# -----------------------------------------------

scene = set_scene()

button(text="Run", pos=scene.title_anchor, bind=Run)
button(text="Reset M", pos=scene.title_anchor, bind=Reset_M) 
button(text="  RF  ", pos=scene.title_anchor, bind=Pulse_start)
button(text="Lab frame", pos=scene.title_anchor, bind=Rotate_frame)

scene.append_to_caption('\n RF <i>H</i><sub>1</sub> amplitude:\n')
sl_h1 = slider(min=0, max=0.4, value=0.05, length=250, bind=Set_h1, left=25, right=15)
wt_h1 = wtext(text='{:1.3f}'.format(sl_h1.value))
scene.append_to_caption(' <i>H</i><sub>0</sub>')

scene.append_to_caption('\n RF detuning &Delta;<i>&omega;</i>:\n')
sl_dW = slider(min=-1.0, max=1.0, value=0.0, length=250, bind=Set_Delta_omega, left=25, right=15)
wt_dW = wtext(text='{:1.3f}'.format(sl_dW.value))
scene.append_to_caption(' &omega;<sub>0</sub>') 

scene.append_to_caption('\n Static field &Delta;<i>H</i><sub>0</sub> shift:\n')
sl_h0 = slider(min=-1.0, max=1.0, value=0.0, length=250, bind=Set_h0, left=25, right=15)
wt_h0 = wtext(text='{:1.3f}'.format(sl_h0.value))
scene.append_to_caption(' <i>H</i><sub>0,static</sub>') 

scene.append_to_caption(' \n <i>H</i><sub>1</sub> type: ')
RF_menu = menu( choices=['rotating','linear'], bind=RF_select )

scene.append_to_caption(' \n <i>T</i><sub>1</sub> (half-life): ')
T_menu = menu( choices=['4.0','10.0','50.0','100.0','1000.0', '10000.0'], bind=T_select )

scene.append_to_caption(' \n\n')

gd = graph(fast=True, width=500, height=150, title='Pickup coil signal',
            ytitle='Amplitude', xtitle='Time')
f1 = gcurve(interval=ticks_per_frame, color=color.blue, width=1)
button(text="Clear", bind=Clear_plot)
button(text="Start plot", bind=Stop_plot)

draw_axes()

In [ ]:
# Initial values of loop variables
phi = 0.0 # cumulative phase of m
dphi = 0.0 # increment of phi
phi1 = omega()*t # phase of RF
dphi1 = omega()*dt # increment of phi1
h = h0 # initial field vector
m = m0 # initial magnetization
dm = vector(0.,0.,0.) # initial magnetization increment 
# 
# Initial control states
running = False
rotating = False
pulsing = False
RFtype='rotating'
plotting=False
T = T0/log(2) # relaxation time (T1 or T2)
DH0 = 0.0 # static field sweep

# Main loop
while True:
    rate(ticks_per_s)
    h.z = (1.0 + sl_h0.value)*h_static_0
    if pulsing:
        h.x, h.y = RF_function( phi1, RFtype )
    else:
        h.x = h.y = 0.0
    if rotating:
        B.axis = h - vector(0,0,omega()/gamma)
    else:
        B.axis = h
    M.axis = m
    if running:
        if rotating:
            scene.camera.rotate(angle = dphi1, axis=vector(0,0,1), origin=vector(0,0,0))
        dm_mid = -(gamma*cross(h,m) + (m - chi*h)/T)*0.5*dt # Using estimated midpoint to calculate arrow
        dm = -(gamma*cross(h,m+dm_mid) + (m+dm_mid - chi*h)/T)*dt
        m = m + dm
        dphi = -gamma*h.mag*dt
        phi = phi + dphi
        dphi1 = -omega()*dt
        phi1 = phi1 + dphi1
        t = t + dt
        m_signal = dm.x/dt
        if plotting: f1.plot([t , m_signal])